# <font color='blue'> PPHA 30546 Machine Learning | PS3</font> 

Kathy Zhang, Yimeng Wu

In [1]:
# Import the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split
from sympy import symbols, Eq, solve, exp

from IPython.display import Image
from IPython.core.display import HTML 

import warnings
warnings.filterwarnings('ignore')

###  CH5 Q6
#### 6. We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the sm.GLM() function. Do not forget to set a random seed before beginning your analysis. 

• In parts (a) and (d), the problem references the sm.glm() function. You can use the sm.Logit() function instead if you’d prefer. The sm.glm() function is more general way to estimate different linear models (hence the GLM name), but it will estimate a logistic regression with the appropriate arguments.

• Part (a) also references the summarize() function. This is a typo. The authors mean the .summary() method as a way to view the results of a sm.glm() fit.

• In part (c), please draw 1,000 bootstrap samples when bootstrapping your standard errors.

#### (a) Using the summarize() and sm.GLM() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors.

#### (b) Write a function, boot_fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model.

#### (c) Following the bootstrap example in the lab, use your boot_fn() function to estimate the standard errors of the logistic regression coefficients for income and balance.

#### (d) Comment on the estimated standard errors obtained using the sm.GLM() function and using the bootstrap.

<font color='blue'> aaa

###  CH5 Q8

#### 8. We will now perform cross-validation on a simulated data set.

• In part (a), note that the code you’re given sets a random seed equal to 1.

• In part (c), please keep the same random seed as in part (a).

• In part (d), please set a random seed equal to 2.

#### (a) Generate a simulated data set as follows:
`rng = np.random.default_rng(1)`

`x = rng.normal(size=100)`

`y = x - 2 * x**2 + rng.normal(size=100)`

#### In this data set, what is n and what is p? Write out the model used to generate the data in equation form.

#### (b) Create a scatterplot of X against Y . Comment on what you find.

#### (c) Set a random seed, and then compute the LOOCV errors that result from fitting the following four models using least squares:

i. Y = B0 + B1X + e

ii. Y = B0 + B1X + B2X^2 + e

iii. Y = B0 + B1X + B2X^2 + B3X^3 + e

iv. Y = B0 + B1X + B2X^2 + B3X^3 + B4X^4 + e.

#### Note you may find it helpful to use the data.frame() function to create a single data set containing both X and Y .

#### (d) Repeat (c) using another random seed, and report your results. Are your results the same as what you got in (c)? Why?

#### (e) Which of the models in (c) had the smallest LOOCV error? Is this what you expected? Explain your answer.

#### (f) Comment on the statistical significance of the coefficient estimates that results from fitting each of the models in (c) using least squares. Do these results agree with the conclusions drawn based on the cross-validation results?

###  CH6 Q11

#### 11. We will now try to predict per capita crime rate in the Boston dataset.

• In part (a), use forward stepwise & backwards stepwise selection (FSS & BSS) instead of the methods the book lists. Do so based on using a mathematical adjustment approach (AIC) and 5-Fold Cross-Validation (5FCV) to estimate the test error. Use the entire dataset for 5FCV, shuffle the data randomly for splitting, and set random_state=23). This means you will select a model four different ways: FSS-AIC, FSS-5FCV, BSS-AIC, BSS-5FCV.

• As part of your answer for part (b), be sure to explain why the the different methods you use may select different models.



#### (a) Try out some of the regression methods explored in this chapter, such as best subset selection, the lasso, ridge regression, and PCR. Present and discuss results for the approaches that you consider.

#### (b) Propose a model (or set of models) that seem to perform well on this data set, and justify your answer. Make sure that you are evaluating model performance using validation set error, crossvalidation, or some other reasonable alternative, as opposed to using training error.

#### (c) Does your chosen model involve all of the features in the dataset? Why or why not?